# Uncapacitated facility location problem

## Problem description

The uncapacitated facility location problem (UFLP) is a well-known problem in combinatorial optimization. It is a problem that arises in many applications, such as supply chain management, urban planning, and data clustering. The problem can be described as follows:

- There are a set of facilities and a set of clients. Each facility has an opening cost, and each client has a demand and a cost to be served from each facility.
- The goal is to decide which facilities to open and how to assign clients to open facilities in order to minimize the sum of the opening costs and the service costs.

The problem can be formulated as an integer programming problem, and it is known to be NP-hard. In this notebook, we present a simple model for the UFLP and solve it using the Gurobi Python API.


## Mathematical formulation

The uncapacitated facility location problem can be formulated as the following integer programming problem:

\begin{equation}
\text{minimize} \quad \sum_{i \in F} f_i y_i + \sum_{i \in F} \sum_{j \in C} c_{ij} x_{ij}
\end{equation}

\begin{equation}
\text{subject to} \quad \sum_{i \in F} x_{ij} = 1, \quad \forall j \in C
\end{equation}

\begin{equation}

x_{ij} \leq y_i, \quad \forall i \in F, \forall j \in C
\end{equation}

\begin{equation}
x_{ij} \in \{0, 1\}, \quad \forall i \in F, \forall j \in C
\end{equation}

\begin{equation}
y_i \in \{0, 1\}, \quad \forall i \in F
\end{equation}

Where:

- $F$ is the set of facilities
- $C$ is the set of clients
- $f_i$ is the cost of opening facility $i$
- $c_{ij}$ is the cost of serving client $j$ from facility $i$
- $x_{ij}$ is a binary variable that is equal to 1 if client $j$ is assigned to facility $i$, and 0 otherwise
- $y_i$ is a binary variable that is equal to 1 if facility $i$ is opened, and 0 otherwise



In [78]:
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [18]:
16//7

2

In [16]:
# Helper function to extract 100 elements from multiple lines (12 per line)
def extract_full_row(start_idx, lines_needed, lines):
    elements = []
    for i in range(lines_needed):
        line_elements = list(map(float, lines[start_idx + i].split()))
        elements.extend(line_elements)
    return elements

In [65]:
def read_ufl_file(filename):

    """
    n - number of clients
    m - number of facilities
    fixed_cost - fixed cost of opening a facility
    cost_matrix - cost of connecting a client to a facility
    """
    with open(filename, 'r') as f:
        lines = f.read().splitlines()
        # print(lines)
        fixed_cost = []
        cost_matrix = []
        m, n = map(int, lines[0].split())
        print(n, m)

        lines_reqd = m // 7 + 1
        # print(lines_reqd)

        fixed_cost_start_index = 1
        fixed_cost_end_index = m + 1
        init_cost_matrix_start_index = fixed_cost_end_index + 1

        # fixed cost
        for i in range(fixed_cost_start_index, fixed_cost_end_index):
            fixed_cost.append(float(lines[i].split()[1]))
        
        # cost matrix
        for i in range(n):
            cost_matrix_start_index = init_cost_matrix_start_index + i * (lines_reqd + 1)
            cost_matrix.append(extract_full_row(cost_matrix_start_index, lines_reqd, lines))
        # print(cost_matrix)
        
        return n, m, fixed_cost, cost_matrix    


    # return n, m, f, c


# Weaker Formulation

Instead of using

\begin{equation}

x_{ij} \leq y_i, \quad \forall i \in F, \forall j \in C
\end{equation}

we can use

\begin{equation}
\sum_{j \in C} x_{ij} \leq |C| y_i, \quad \forall i \in F
\end{equation}


In [116]:
def solve_ufl_weak(n, m, fixed_cost, cost_matrix, gap, output_file, M = 1000):
    # Create a new model
    model = gp.Model("ufl")

    # Create variables
    y = model.addVars(m, vtype=GRB.BINARY, name="y")
    x = model.addVars(n, m, vtype=GRB.BINARY, name="x")

    # Set objective
    model.setObjective(gp.quicksum(fixed_cost[j] * y[j] for j in range(m)) + gp.quicksum(cost_matrix[i][j] * x[i, j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

    # Add constraints
    model.addConstrs((x.sum(i, '*') == 1 for i in range(n)), name="c1")
    model.addConstrs((sum(x[i, j] for i in range(n)) <= M * y[j] for j in range(m)), name="c2")

    # Optimize model
    # model.Params.MIPGap = gap
    model.optimize()

    # Print solution
    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % model.objVal)

    # write lp file and convert binary variables to continuous
    for var in model.getVars():
        print(var.varName)
        var.vtype = GRB.CONTINUOUS  # Convert to continuous
        # var.lb = 0  # Ensure lower bound is >= 0
        # var.ub = 1  # Ensure upper bound is <= 1

    # Write the modified model with continuous variables to an .lp file
    model.update()  # Ensure the model reflects changes to the variable types
    model.write(output_file)
    # return model


# Stronger Formulation





In [112]:
def solve_ufl_strong(n, m, fixed_cost, cost_matrix, gap, output_file):
    # Create a new model
    model = gp.Model("ufl")

    # Create variables
    y = model.addVars(m, vtype=GRB.BINARY, name="y")
    x = model.addVars(n, m, vtype=GRB.BINARY, name="x")

    # Set objective
    model.setObjective(gp.quicksum(fixed_cost[j] * y[j] for j in range(m)) + gp.quicksum(cost_matrix[i][j] * x[i, j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

    # Add constraints
    model.addConstrs((x.sum(i, '*') == 1 for i in range(n)), name="c1")
    model.addConstrs((x[i, j] <= y[j] for i in range(n) for j in range(m)), name="c2")

    # Optimize model
    model.Params.MIPGap = gap
    model.optimize()

    # Print solution
    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % model.objVal)

    # write lp file and convert binary variables to continuous
    for var in model.getVars():
        print(var.varName)
        var.vtype = GRB.CONTINUOUS  # Convert to continuous
        # var.lb = 0  # Ensure lower bound is >= 0
        # var.ub = 1  # Ensure upper bound is <= 1

    # Write the modified model with continuous variables to an .lp file
    model.update()  # Ensure the model reflects changes to the variable types
    model.write(output_file)
    # return model

# Reading the data

In [107]:
output_dir = 'lp_files_UFL_Strong'
os.makedirs(output_dir, exist_ok=True)

In [117]:
for root, dirs, files in os.walk("UFL"):
        for file in files:
                print(file)
                n, m, f, c = read_ufl_file(os.path.join(root, file))
                solve_ufl_strong(n, m, f, c, 0.01, os.path.join(output_dir, f"{file}.lp"))
                solve_ufl_weak(n, m, f, c, 0.01, os.path.join(output_dir, f"{file}.lp"), 1000)
                break


cap101.txt
50 25
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1300 rows, 1275 columns and 3750 nonzeros
Model fingerprint: 0xa2c9e121
Variable types: 0 continuous, 1275 integer (1275 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2730264.7125
Presolve removed 1169 rows and 1156 columns
Presolve time: 0.01s
Presolved: 131 rows, 119 columns, 321 nonzeros
Found heuristic solution: objective 812193.42500
Variable types: 0 continuous, 119 integer (119 binary)
Found heuristic solution: objective 804468.33750

Root relaxation: interrupted, 38 iterations, 0.00 seconds (0.00 work units)

